# Лабораторная работа №3. Реализация сверточной нейронной сети

**Данные:** В работе предлагается использовать набор данных notMNIST, который состоит из изображений размерностью 28×28 первых 10 букв латинского алфавита (A … J, соответственно). Обучающая выборка содержит порядка 500 тыс. изображений, а тестовая – около 19 тыс.

**Данные можно скачать по ссылке:**
- https://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz (большой набор данных);
- https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz (маленький набор данных);

Описание данных на английском языке доступно по ссылке:
- http://yaroslavvb.blogspot.sg/2011/09/notmnist-dataset.html


In [1]:
import tensorflow as tf
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random
import os

In [2]:
import pathlib

train_data_dir = pathlib.Path('./Datasets/1-3/notMNIST_large')

train_image_count = len(list(train_data_dir.glob('*/*.png')))
train_image_count

529113

In [3]:
test_data_dir = pathlib.Path('./Datasets/1-3/notMNIST_small')
test_image_count = len(list(test_data_dir.glob('*/*.png')))
test_image_count

18724

In [4]:
BATCH_SIZE = 1000
IMG_HEIGHT = 28
IMG_WIDTH = 28
CLASS_NAMES = np.array([item.name for item in train_data_dir.glob('*')])
CLASS_NAMES

array(['H', 'F', 'E', 'I', 'D', 'B', 'A', 'J', 'C', 'G'], dtype='<U1')

In [5]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.02)
train_data_gen = train_image_generator.flow_from_directory(directory=str(train_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     color_mode='grayscale',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

Found 529113 images belonging to 10 classes.


In [6]:
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_gen = train_image_generator.flow_from_directory(directory=str(test_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     color_mode='grayscale',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

Found 18724 images belonging to 10 classes.


In [19]:
import random
import datetime

class Model:
    EPOCH_COUNT = 10
    COUNT_OF_BUTCHES = 530
    SHUFFLE_BUFFER_SIZE = 100
    ACTIVATION_FUNCTIONS = (
        'linear',
        'sigmoid', 
        'tanh',
        'relu',
    )
    
    def __init__(self, layers=[], optimizer=None):
        self.layers = layers
        self.optimizer = optimizer or tf.keras.optimizers.SGD(
            learning_rate=0.1
        )

    def initialize(self):
        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

        self.model = tf.keras.models.Sequential(self.layers)

        self.model.compile(optimizer=self.optimizer, loss=loss_fn, metrics=['accuracy'])

    def fit(self, x_train, y_train=None, **kwargs):
        return self.model.fit(x_train,  y_train, epochs=self.EPOCH_COUNT, validation_split=0.05, **kwargs)

    def fit_generator(self, generator, batch_count=None):
        batch_count = batch_count or self.COUNT_OF_BUTCHES

        now = datetime.datetime.now()
        tran_accuracy = 0
        val_accuracy = 0

        for i in range(batch_count):
            x, y = next(generator)
            batch_history = self.fit(x, y, verbose=0)
            tran_accuracy += batch_history.history['accuracy'][-1]
            val_accuracy += batch_history.history['val_accuracy'][-1]

        print('\nSpent time:', datetime.datetime.now() - now)
        print('Average train error:', tran_accuracy / batch_count)
        print('Average validation error:', val_accuracy / batch_count)
        return [tran_accuracy, val_accuracy]

    def test(self, x_test, y_test=None):
        return self.model.evaluate(x_test,  y_test, verbose=1)
    
    def summary(self):
        return self.model.summary()

### Задание 1. Реализуйте нейронную сеть с двумя сверточными слоями, и одним полносвязным с нейронами с кусочно-линейной функцией активации. Какова точность построенное модели?


In [20]:
# first Conv layer is input one

layers = [
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='sigmoid')
]

In [21]:
def test_model(layers):
    model = Model(layers)
    model.initialize()
    print(model.summary(), '\n')
    
    model.fit_generator(train_data_gen)
    
    model.test(test_data_gen)

In [22]:
test_model(layers)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
flatten_4 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                216330    
Total params: 225,898
Trainable params: 225,898
Non-trainable params: 0
_________________________________________________________________
None 


Spent time: 0:45:12.965865
Average train error: 0.881724117049631
Average validation error: 0.8686792408520321
  ...
    to  
  ['...']
19/19 [==============================] - 4s 187ms/step - loss: 1.5287 - accuracy: 0.9384


Данная модель показывала следующую точность:
- обучающая выборка: 0.88
- валидационная выборка: 0.86
- тестовая выборка: 0.94

### Задание 2. Замените один из сверточных слоев на слой, реализующий операцию пулинга (Pooling) с функцией максимума или среднего. Как это повлияло на точность классификатора?


In [24]:
layers = [
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='sigmoid')
]

In [25]:
test_model(layers)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________
None 


Spent time: 0:14:58.653167
Average train error: 0.8712481779872246
Average validation error: 0.8601383593847167
  ...
    to  
  ['...']
19/19 [==============================] - 3s 139ms/step - loss: 1.5283 - accuracy: 0.9356


Точность классификатора почти не изменилась (слегка понизилась), но скорость обучения значительно повысилась.

### Задание 3. Реализуйте классическую архитектуру сверточных сетей LeNet-5 (http://yann.lecun.com/exdb/lenet/).

In [26]:
layers = [
    tf.keras.layers.Conv2D(32, 6, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.Conv2D(32, 6, activation='relu'),
    tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=120, activation='relu'),
    tf.keras.layers.Dense(units=84, activation='relu'),
    tf.keras.layers.Dense(units=10, activation = 'softmax'),
]

In [27]:
test_model(layers)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 23, 23, 32)        1184      
_________________________________________________________________
average_pooling2d_2 (Average (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 32)          36896     
_________________________________________________________________
average_pooling2d_3 (Average (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 120)               34680     
_________________________________________________________________
dense_9 (Dense)              (None, 84)               

### Задание 4. Сравните максимальные точности моделей, построенных в лабораторных работах 1-3. Как можно объяснить полученные различия?


Лучше всего показали себя **сверточные нейронные сети** с точностью в **0.93-0.94**. Это можно объяснить выделением действительно важных признаков в изображениях и обучать нейронную сеть на соответсвиях этих признаков и результате.

Следующий результат показала **модель с полносвязанными нейроннами** ее наибольшая точность составила **0.91**. Полносвязанные нейронные сети также выделяют важные признаки в изображении, однако из-за огромного количества связей между нейронаними, важные признаки могут размываться, а второстепенные наоборот более честко выделяться - это непрерывно ведет к переобучению. Получение более точных результатов непрерывно требует идеального подбора параметров регуляризации (l1, l2, dropout).

**Простой классификатор** показал точность **0.835**. Что говорит о том, что классификатор в общем неплох, но ввиду того, что классификатор не выделяет важные/не важные признаки, то для хороших результатов требуется большое количество данных, при этом так же предется следить за переобучением.